### Section 0: Loading packages and testing earth engine connection

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ee
import ccblc
import folium

In [ ]:
%reload_ext ccblc

In [ ]:
# authenticate earth engine api access (not required on every run)
ee.Authenticate()

In [ ]:
# initialize earth engine api access (required on every run)
ee.Initialize()

In [ ]:
# test whether the initialization was successful by printing the elevation of Mount Everest
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

### Section 1: unmixing one image

In [ ]:
# select the sensor to use
sensor = 'Landsat8'
collectionName = ccblc.getCollection(sensor)
bands = ccblc.getBands(sensor)
scaler = ccblc.getScaler(sensor)
maskFunction = ccblc.Mask.bySensor(sensor)
visBands = ["B6", "B5", "B4"]

# set the dates
startDate = "2020-07-01";
endDate = "2020-11-01";

# get a land mask to use
land = ee.Image("UMD/hansen/global_forest_change_2019_v1_7").select(["datamask"]).eq(1)

# create and filter the image collection
collection = ee.ImageCollection(collectionName).filterDate(startDate, endDate).map(maskFunction).select(bands)

In [ ]:
# create a composite
sr = collection.median().multiply(scaler).updateMask(land)

# map it
folium.Map.addLayer = ccblc.Map.addLayer

# Set visualization parameters.
visParams = {
  'min': 0,
  'max': 0.35,
  'bands': visBands
}

# Create a folium map object.
Map = folium.Map(location=[37.5, -118], zoom_start=6, height=500)

# Add the image to the map object.
Map.addLayer(sr, visParams, 'S2')

# Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

# Display the map.
display(Map)

now we'll actually unmix the image using the first spectra from each bundle

In [ ]:
# initialize the spectral bundle selection
ccblc.Unmix.setSensor(sensor)

# unmix the thing
unmixed = ccblc.Unmix.SVN(sr).updateMask(land)

# map it
visParams = {'min': [0, 0.1, 0], 'max': [0.8, 1, 0.8]}
Map = folium.Map(location=[37.5, -118], zoom_start=6, height=500)
Map.addLayer(unmixed, visParams, 'unmixed')
Map.add_child(folium.LayerControl())
display(Map)